# Learn v75 med walkthrough-metoden

In [1]:
import pandas as pd 
import numpy as np 
from catboost import CatBoostClassifier,Pool,cv,utils

import sys
sys.path.append('C:\\Users\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel')
import V75_scraping as vs

In [2]:
### returnera en modell med parametrar satta
def get_model(d=6,l2=2,iterations=3000,use_best=True,verbose=False):
    model = CatBoostClassifier(iterations=iterations,use_best_model=use_best, 
        custom_metric=['Logloss', 'AUC','Recall', 'Precision', 'F1', 'Accuracy'],

        eval_metric='Accuracy', 
        depth=d,l2_leaf_reg=l2,
        auto_class_weights='Balanced',verbose=verbose, random_state=2021) 
    return model                

In [3]:
### Features som inte används vid träning
def remove_features(df,remove_mer=[]):
    #remove_mer=['h5_perf','h5_auto','h4_perf','h4_auto', 'h3_perf', 'h2_perf']
    df.drop(['avd','startnr','vodds','podds','bins','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat'],axis=1,inplace=True) #
    if remove_mer:
        df.drop(remove_mer,axis=1,inplace=True)
    
    # df=check_unique(df.copy())
    # df=check_corr(df.copy())
    return df

In [4]:
 ## byt ut alla NaN till text för cat_features
def replace_NaN(X_train,X_test=None, cat_features=[]):
    # print('cat_features',cat_features)
    for c in cat_features:
        # print(c)
        X_train.loc[X_train[c].isna(),c] = 'None'       ### byt ut None-värden till texten 'None
        if X_test is not None:  ## om X_test är med
            X_test.loc [X_test[c].isna(),c] = 'None'    ### byt ut None-värden till texten 'None

    return X_train,X_test

In [5]:
def scrape_nya_lopp():
    nya_lopp,strukna = vs.v75_scraping(resultat=True,history=True)

    df=pd.concat([pd.read_csv('all_data.csv'), nya_lopp])
    print('shape med nya lopp',df.shape)
    #ta bort dubletter
    df.drop_duplicates(['datum','avd','häst'],inplace=True)
    df.sort_values(by=['datum','avd'],inplace=True)
    df.reset_index(drop=True,inplace=True)
    print('shape med dubletter bort',df.shape)

    df.to_csv('all_data.csv', index=False)

    print("första datum i df =",df.datum.head(1).to_list()[0])
    print("sista  datum i df =",df.datum.tail(1).to_list()[0])

    return df,nya_lopp

In [6]:
### beräkna vilka datum att använda ###
def get_alla_datum(proc=0.75, total_omlärning = False):
    if total_omlärning:
        nya_lopp=None
        df = pd.read_csv('all_data.csv')     
        alla_datum = df.datum.unique()
        split_ix = int(len(alla_datum)*proc)
    else:
        # normalt adderar vi bara 1 eller flera veckor från "omg_att_spela_link.csv"
        df, nya_lopp = scrape_nya_lopp()  # scrape från 'omg_att_spela_link.csv' och addera till df
        omg_df = pd.read_csv('omg_att_spela_link.csv')     
        startix=omg_df.Link.str.find('spel')[0]    # index till 'spel' i url
        alla_datum = omg_df.Link.str.slice(start=startix+5,stop=startix+15).to_list() # en datum 
        split_ix=0
        print(f'datum att lära från {alla_datum}')

    return df,nya_lopp,alla_datum,split_ix


## Walkthrough-funktionen  här

In [7]:

### Kör en walkthrough learn här, en datum i taget framåt

# Jag har ändrat till att alla steg kör utan test-datam ed fast iterations=100
def walkthrough(classic_test=False, verbose=False):
    
    df, nya_lopp, alla_datum, split_ix = get_alla_datum()

    l2_leaf_regs=2
    model=get_model(use_best=False,iterations=100)
    df=remove_features(df.copy())
    cat_features = list(df.loc[:,df.dtypes=='O'].columns)
    df,_ = replace_NaN(df.copy(), cat_features=cat_features)    
    print(f'cat_features {cat_features}\n')

    df['plac']=(df.plac==1)*1
        
    for nr,datum in enumerate(alla_datum[split_ix:]):
        print(f'walk-iter {nr+1} av {len(alla_datum[split_ix:])} ',end=': ')

        X_train = df.loc[df.datum<datum,:].copy()
        y_train = X_train.plac; X_train.drop(['plac'],axis=1,inplace=True)

        if classic_test:    ### klassisk train/test utan walkthrough
            X_test  = df.loc[df.datum>=datum,:].copy()
            y_test  = X_test.plac;  X_test.drop(['plac'],axis=1,inplace=True)
            train_pool = Pool(X_train,y_train,cat_features=cat_features)
            test_pool = Pool(X_test,y_test,cat_features=cat_features)
            model.fit(train_pool,use_best_model=True, verbose=verbose,eval_set=test_pool)
        else:
            X_test  = df.loc[df.datum==datum,:].copy()
            y_test  = X_test.plac;  X_test.drop(['plac'],axis=1,inplace=True)
            train_pool = Pool(X_train,y_train,cat_features=cat_features)
            test_pool = Pool(X_test,y_test,cat_features=cat_features)
            model.fit(train_pool,use_best_model=False, verbose=verbose)

        print('best iteration',model.get_best_iteration(), '\tbest score', round(model.get_best_score()['learn']['Accuracy'],3) )
        ##['validation']['Logloss'],3),'\t', round(model.get_best_score()['validation']['Accuracy:use_weights=true'],3))
        
        if classic_test:    ### klassisk train/test utan walkthrough
            return model,cat_features
    
        model.save_model('modeller/model_'+datum)

    X_train =df.copy().drop('plac',axis=1)
    y_train = df.plac 
    model.fit(X_train,y=y_train,cat_features=cat_features)
    print(f'spara model_senaste',datum)
    model.save_model('modeller/model_senaste')

    return df,nya_lopp, model,cat_features

### Här körs hela walkthrough

In [8]:
df, nya_lopp, model, cat_features = walkthrough(classic_test=False, verbose=False)


omgång 1: https://www.atg.se/spel/2021-09-11/V75/
klickade på ANPASSA
anpassa klar - break
ant resultat 7
ant lopp 7


2021-09-12 10:14:53.940 WARNING root: 2021-09-11 avd: <selenium.webdriver.remote.webelement.WebElement (session="5ff274cd575aff156ac53b6fec93316b", element="4868221f-21d8-40a8-a4a3-c7263afbae3d")>


Ant priser 7
ant names,vodds,podds,rader,streck 12 12 12 12


2021-09-12 10:14:54.353 WARNING root: 0


AVD 1 BOLLNÄS 2640 VOLTSTART 

...

.

2021-09-12 10:16:45.822 WARNING root: 10


.

2021-09-12 10:16:47.160 WARNING root: 11


..

det tog 115.098 sekunder
utdelning: 27842, 318, 38
startar Fixa mer
tog bort 7 strukna från 89 till 82
rensade totalt bort 7 hästar i städa_och_rensa. Från 89 till 82
shape med nya lopp (41845, 79)
shape med dubletter bort (41845, 79)
första datum i df = 2014-12-28
sista  datum i df = 2021-09-11
datum att lära från ['2021-09-11']
cat_features ['datum', 'bana', 'häst', 'kusk', 'kön', 'h1_kusk', 'h1_bana', 'h2_kusk', 'h2_bana', 'h3_kusk', 'h3_bana', 'h4_kusk', 'h4_bana', 'h5_kusk', 'h5_bana']

walk-iter 1 av 1 : best iteration None 	best score 0.75
spara model_senaste 2021-09-11


## Kör allt ovanför walkthrough
### Se till att "omg_att_spela_link.csv" är ifylld

In [9]:
from catboost.utils import eval_metric
df = pd.read_csv('all_data.csv')     
print(df.columns)
dfval=remove_features(df.copy())
cat_features = list(dfval.loc[:,dfval.dtypes=='O'].columns)
dfval,_ = replace_NaN(dfval.copy(), cat_features=cat_features)    
    
validation = nya_lopp.copy()
validation = validation[dfval.columns]
# validation.drop('startnr',axis=1,inplace=True)
# validation = remove_features(validation)
y=validation.plac
y=(y==1)*1
validation.drop('plac',axis=1,inplace=True)
# cat_features = validation.loc[:,validation.dtypes=='O'].columns
        
val_pool=Pool(validation,y,cat_features)
yhat=model.predict(val_pool)
print(utils.get_confusion_matrix(model,val_pool))
eval_metric(yhat,y,'Accuracy')



Index(['datum', 'avd', 'bana', 'häst', 'kusk', 'streck', 'vodds', 'podds',
       'kr', 'spår', 'dist', 'lopp_dist', 'start', 'ålder', 'kön', 'plac',
       'pris', 'h1_dat', 'h1_kusk', 'h1_bana', 'h1_spår', 'h1_plac', 'h1_pris',
       'h1_odds', 'h1_kmtid', 'h2_dat', 'h2_kusk', 'h2_bana', 'h2_spår',
       'h2_plac', 'h2_pris', 'h2_odds', 'h2_kmtid', 'h3_dat', 'h3_kusk',
       'h3_bana', 'h3_spår', 'h3_plac', 'h3_pris', 'h3_odds', 'h3_kmtid',
       'h4_dat', 'h4_kusk', 'h4_bana', 'h4_spår', 'h4_plac', 'h4_pris',
       'h4_odds', 'h4_kmtid', 'h5_dat', 'h5_kusk', 'h5_bana', 'h5_spår',
       'h5_plac', 'h5_pris', 'h5_odds', 'h5_kmtid', 'h1_dist', 'h2_dist',
       'h3_dist', 'h4_dist', 'h5_dist', 'bins', 'h1_auto', 'h2_auto',
       'h3_auto', 'h4_auto', 'h5_auto', 'h1_perf', 'h2_perf', 'h3_perf',
       'h4_perf', 'h5_perf', 'senast', 'delta1', 'delta2', 'delta3', 'delta4',
       'startnr'],
      dtype='object')
[[53. 22.]
 [ 1.  6.]]


[0.7195121951219512]

In [10]:
validation.datum.max()

'2021-09-11'

In [11]:
model.get_feature_importance(prettified=True).head(30)

,Feature Id,Importances
0,streck,61.110261
1,datum,8.303840
2,häst,4.938673
3,h2_perf,1.503465
4,h5_odds,1.368404
5,h1_plac,1.198600
6,h1_odds,1.180801
7,h1_bana,1.173274
8,h2_odds,0.834960
9,h4_odds,0.819075


In [12]:
model.get_params()

{'iterations': 100,
 'depth': 6,
 'l2_leaf_reg': 2,
 'use_best_model': False,
 'verbose': False,
 'auto_class_weights': 'Balanced',
 'custom_metric': ['Logloss', 'AUC', 'Recall', 'Precision', 'F1', 'Accuracy'],
 'eval_metric': 'Accuracy',
 'random_state': 2021}

# Kör cross validation

## init  - kör först allt t.o.m 'replace_NaN()' ovan

In [13]:
model = get_model().load_model('modeller/model_senaste')
df = pd.read_csv('all_data.csv')     
# print(df.columns)
df=remove_features(df.copy())
cat_features = list(df.loc[:,df.dtypes=='O'].columns)
df,_ = replace_NaN(df.copy(), cat_features=cat_features)    
y=df.plac
y=(y==1)*1
df.drop('plac',axis=1,inplace=True)


In [14]:
df[df.columns[(df.dtypes=='object').values.tolist()]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41845 entries, 0 to 41844
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   datum    41845 non-null  object
 1   bana     41845 non-null  object
 2   häst     41845 non-null  object
 3   kusk     41845 non-null  object
 4   kön      41845 non-null  object
 5   h1_kusk  41845 non-null  object
 6   h1_bana  41845 non-null  object
 7   h2_kusk  41845 non-null  object
 8   h2_bana  41845 non-null  object
 9   h3_kusk  41845 non-null  object
 10  h3_bana  41845 non-null  object
 11  h4_kusk  41845 non-null  object
 12  h4_bana  41845 non-null  object
 13  h5_kusk  41845 non-null  object
 14  h5_bana  41845 non-null  object
dtypes: object(15)
memory usage: 4.8+ MB


## cv

In [18]:

cv_pool = Pool(df,y,cat_features=cat_features)

params = {
         'use_best_model': True,
         'eval_metric' : 'AUC',
         "loss_function": "Logloss",
         'early_stopping_rounds': 100,
         'verbose': 50,
}

cv_score =cv(pool=cv_pool, 
   params=params, 
   dtrain=None, 
   iterations=2000, 
   num_boost_round=None,
   fold_count=5, 
   nfold=None,
   inverted=False,
   partition_random_seed=0,
   seed=2021, 
   shuffle=False, 
   logging_level=None, 
   stratified=True,
   as_pandas=True,
   type='TimeSeries')

0:	test: 0.7143153	best: 0.7143153 (0)
50:	test: 0.8080923	best: 0.8080923 (50)
100:	test: 0.8113438	best: 0.8115399 (79)
150:	test: 0.8110944	best: 0.8115399 (79)
Stopped by overfitting detector  (100 iterations wait)


In [20]:
cv_score

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.714315,0.014453,0.659024,0.000214,0.658928,0.000237
1,1,0.718921,0.014591,0.629223,0.001989,0.629017,0.002093
2,2,0.730108,0.009589,0.600586,0.001428,0.600293,0.001403
3,3,0.739781,0.015526,0.574462,0.001847,0.573974,0.001891
4,4,0.749625,0.023369,0.551366,0.002391,0.550731,0.002558
...,...,...,...,...,...,...,...
175,175,0.810337,0.006610,0.240060,0.002394,0.208227,0.005777
176,176,0.810366,0.006638,0.240047,0.002393,0.208103,0.005814
177,177,0.810357,0.006678,0.240051,0.002423,0.207968,0.005907
178,178,0.810353,0.006811,0.240094,0.002511,0.207843,0.005931


In [31]:
from IPython.display import display
print(df.datum.max())
display(cv_score[cv_score['test-Logloss-mean'].min() == cv_score['test-Logloss-mean']])
display(cv_score[cv_score['test-AUC-mean'].max() == cv_score['test-AUC-mean']])

2021-09-11


,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
128,128,0.810912,0.005819,0.239738,0.001498,0.213718,0.00374


,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
79,79,0.81154,0.0063,0.243613,0.001405,0.224876,0.001559


In [28]:
help(display)

Help on function display in module IPython.core.display:

display(*objs, include=None, exclude=None, metadata=None, transient=None, display_id=None, **kwargs)
    Display a Python object in all frontends.
    
    By default all representations will be computed and sent to the frontends.
    Frontends can decide which representation is used and how.
    
    In terminal IPython this will be similar to using :func:`print`, for use in richer
    frontends see Jupyter notebook examples with rich display logic.
    
    Parameters
    ----------
    *objs : object
        The Python objects to display.
    raw : bool, optional
        Are the objects to be displayed already mimetype-keyed dicts of raw display data,
        or Python objects that need to be formatted before display? [default: False]
    include : list, tuple or set, optional
        A list of format type strings (MIME types) to include in the
        format data dict. If this is set *only* the format types included
        